📘 Assignment: Understanding and Implementing AdaBoost



In this notebook, you will:

1. Use AdaBoost with a built-in library to understand how it works.
2. Implement AdaBoost step-by-step from scratch using PyTorch.
3. Compare the results between the library implementation and your manual one.
4. Reflect on how reweighting and combining weak learners improves performance.



❗❗❗

There is starting code in the cells - check it for errors and correct it as needed. Check for errors in each line of code - it is not guaranteed to be correct.

For each error you find, write a description and correction.

❗❗❗



In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


## 🧩 Step 1: Load and Explore the Dataset

We'll use the **Breast Cancer Wisconsin** dataset — a public dataset with 30 features and binary labels (malignant vs. benign).


In [ ]:
# Load dataset
data = load_breast_cancer()
X = data.data
y = data.target  # 0 = malignant, 1 = benign

print("Features shape:", X.shape)
print("Labels shape:", y.shape)
print("Classes:", data.target_names)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Preview data
pd.DataFrame(X_train[:5], columns=data.feature_names).head()


Features shape: (569, 30)
Labels shape: (569,)
Classes: ['malignant' 'benign']


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,10.32,16.35,65.31,324.9,0.09434,0.04994,0.01012,0.005495,0.1885,0.06201,...,11.25,21.77,71.12,384.9,0.1285,0.08842,0.04384,0.02381,0.2681,0.07399
1,20.18,19.54,133.80,1250.0,0.11330,0.14890,0.21330,0.125900,0.1724,0.06053,...,22.03,25.07,146.00,1479.0,0.1665,0.29420,0.53080,0.21730,0.3032,0.08075
2,10.66,15.15,67.49,349.6,0.08792,0.04302,0.00000,0.000000,0.1928,0.05975,...,11.54,19.20,73.20,408.3,0.1076,0.06791,0.00000,0.00000,0.2710,0.06164
3,13.56,13.90,88.59,561.3,0.10510,0.11920,0.07860,0.044510,0.1962,0.06303,...,14.98,17.13,101.10,686.6,0.1376,0.26980,0.25770,0.09090,0.3065,0.08177
4,11.37,18.89,72.17,396.0,0.08713,0.05008,0.02399,0.021730,0.2013,0.05955,...,12.36,26.14,79.29,459.3,0.1118,0.09708,0.07529,0.06203,0.3267,0.06994


## ⚙️ Step 2: AdaBoost Using a Library

Let's start by using the built-in `AdaBoostClassifier` to see how AdaBoost performs with a simple weak learner.

We'll use a **decision stump** (a tree of depth 1) as the weak classifier.

In [ ]:
# Use a decision stump as base learner
base_learner = DecisionTreeClassifier(max_depth=1)
ada = AdaBoostClassifier(estimator=base_learner, n_estimators=10, random_state=42)

# Train
ada.fit(X_train, y_train)

# Evaluate
y_pred = ada.predict(X_test)
print("Library AdaBoost Accuracy:", accuracy_score(y_test, y_pred))


Library AdaBoost Accuracy: 0.9649122807017544


## 🧮 Step 3: Implementing AdaBoost From Scratch (PyTorch Version)

Now you'll build AdaBoost manually to understand:
- How sample weights are updated,
- How weak learners contribute to the final model,
- How the ensemble improves over time.


In [ ]:
# Convert to tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
y_train_t = 2 * y_train_t - 1  # convert {0,1} → {-1,+1}

n_samples = X_train_t.shape[0]

# Initialize sample weights equally
w = torch.ones(n_samples) / n_samples

print("Initial weight sum:", w.sum().item())


Initial weight sum: 1.0


## 📜 Step 4: AdaBoost Pseudocode

We'll implement AdaBoost following this algorithm:

For t = 1 to T:
1. Train weak classifier h_t using sample weights w_t
2. Compute predictions y_pred_t
3. Compute weighted error: error_t = Σ_i ( w_i * [y_pred_t != y_i] )
4. Compute model weight: α_t = 0.5 * log((1 - error_t) / error_t)
5. Update weights:
w_i ← w_i * exp(-α_t * y_i * y_pred_t)
Normalize w so Σ_i w_i = 1
Final prediction:
sign( Σ_t α_t * h_t(x) )

Fill in the missing lines in steps 4 and 5.

In [ ]:
T = 12  # number of boosting rounds
learners = []
alphas = []

for t in range(T):
    # 1️⃣ Train weak learner
    stump = DecisionTreeClassifier(max_depth=1)
    stump.fit(X_train, y_train, sample_weight=w.numpy())

    # 2️⃣ Predict on training data
    y_pred = stump.predict(X_train)
    y_pred_t = torch.tensor(2*y_pred - 1, dtype=torch.float32)  # convert to {-1,+1}

    # 3️⃣ Compute weighted error
    incorrect = (y_pred_t != y_train_t)
    error_t = torch.sum(w * incorrect.float()).item()

    # 4️⃣ Compute alpha_t
    alpha_t =

    # 5️⃣ Update weights
    w = w *
    w = w / torch.sum(w)  # normalize weights

    learners.append(stump)
    alphas.append(alpha_t)

    print(f"Round {t+1}: error={error_t:.4f}, alpha={alpha_t:.4f}")


SyntaxError: invalid syntax (ipython-input-4242514617.py, line 19)

## 🧾 Step 5: Evaluate Manual AdaBoost

Now we'll use the weak learners and their α values to make final predictions.

Final prediction rule:
$$
H(x) = \text{sign} \left( \sum_{t=1}^{T} \alpha_t \cdot h_t(x) \right)
$$

In [ ]:
# Predict using the ensemble
H = torch.zeros(len(X_test))

for alpha_t, stump in zip(alphas, learners):
    y_pred =
    y_pred = torch.tensor(2*y_pred - 1, dtype=torch.float32)
    H += alpha_t * y_pred

# Final prediction
y_final = torch.sign(H)
y_final[y_final == -1] = 0  # back to {0,1}

manual_acc = (y_final.numpy() == y_test).mean()
print("Manual AdaBoost Accuracy:", manual_acc)


In [ ]:
print("Library AdaBoost Accuracy:", accuracy_score(y_test, ada.predict(X_test)))
print("Manual AdaBoost Accuracy:", manual_acc)


## 🧪 Extending Knowledge

Try each following experiments, and discuss the results with two different classmates for each. Write a 1-2 sentence summary of your discussion.

1. Change `T` (number of rounds) to 5, 50 then 100.
2. Try `max_depth=2` for a stronger weak learner.
3. Visualize the evolution of `error_t` and `alpha_t` across rounds by plotting.
4. Plot sample weights to see how AdaBoost focuses on harder examples.


Bagging

Write a script that generates a random sequence of N elements and creates M bootstrap samples from that sequence
can use `random.randint` and `random.choices`.